In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/'My Drive'/annd/nlp

/content/drive/My Drive/annd/nlp


In [0]:
!pip install mojimoji

In [0]:
import pandas as pd
import re
import os
import torch
import numpy as np

In [0]:
!pip install transformers

In [0]:
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer, AutoTokenizer
import torchtext
from torchtext import data
import torch
import re
import mojimoji

In [0]:
def tokenizer_with_preprocessing(text):
        text = mojimoji.han_to_zen(text)
        text = re.sub('\r', '', text)
        text = re.sub('\n', '', text)
        text = re.sub('　', '', text)
        text = re.sub(' ', '', text)
        text = re.sub(r'[0-9 ０-９]', '0', text)  
        ret = tokenizer.tokenize(text)
        return ret

In [0]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [0]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
Title = data.Field(sequential=True,tokenize=tokenizer.tokenize, lower=True, batch_first=True, init_token='[CLS]', eos_token='[SEP]',pad_token='[PAD]',unk_token='[UNK]')
Category = data.Field(sequential=False, batch_first=True, use_vocab=False)

train, valid, test = data.TabularDataset.splits(path = '.', train='train1.txt',
    validation='valid1.txt', test='test1.txt', format='tsv',
    fields=[('Title', Title), ('Category', Category)])

In [0]:
train_iter = torchtext.data.BucketIterator(dataset=train, batch_size=2, sort_key=lambda x: len(x.Title), shuffle = True)
valid_iter = torchtext.data.BucketIterator(dataset=valid, batch_size=2, sort_key=lambda x: len(x.Title), shuffle = True)
test_iter = torchtext.data.BucketIterator(dataset=test, batch_size=2,  sort_key=lambda x: len(x.Title), shuffle = True)

In [0]:
Title.build_vocab(train, min_freq=1)
Category.build_vocab(train)

In [0]:
print(Title.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f7a60f28048>, {'[UNK]': 0, '[PAD]': 1, '[CLS]': 2, '[SEP]': 3, '.': 4, "'": 5, '-': 6, 'to': 7, ',': 8, 's': 9, 'in': 10, '##s': 11, 'the': 12, 'of': 13, 'for': 14, 'on': 15, 'as': 16, ':': 17, 'update': 18, 'us': 19, 'and': 20, 'a': 21, '1': 22, 'with': 23, 'at': 24, '2': 25, 'is': 26, 'after': 27, 'new': 28, '$': 29, '"': 30, '(': 31, ')': 32, 'up': 33, 'says': 34, 'stocks': 35, 'from': 36, '##an': 37, '3': 38, 'by': 39, '?': 40, 'ka': 41, 't': 42, '##rda': 43, '##shi': 44, '##y': 45, 'china': 46, 'over': 47, ';': 48, '##b': 49, 'her': 50, 'kim': 51, '##x': 52, 'it': 53, 'be': 54, 'euro': 55, '&': 56, '##a': 57, 'first': 58, '##t': 59, 'out': 60, 'shares': 61, 'more': 62, 'e': 63, 'may': 64, '!': 65, '##o': 66, 'ec': 67, 'data': 68, 'fed': 69, 'she': 70, '##e': 71, 'not': 72, 'that': 73, 'about': 74, 'are': 75, 'dollar': 76, '4': 77, 'day': 78, 'will': 79, 'st': 80, 'all': 81, 'deal': 82, 'wall': 83, 'you': 84, 'sales': 85, 'low': 86, 'h

In [0]:
criterion = torch.nn.BCEWithLogitsLoss()

In [0]:
opt = torch.optim.Adam(model.parameters(), lr=0.0001)

In [0]:
EPOCHS = 10

In [0]:
for epoch in range(EPOCHS):
      model.train()
      for i, batch in enumerate(train_iter):
        x, y = batch.Title, batch.Category
        loss, logit = model(input_ids=x.to(device), labels=y.to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('\rEpoch: %d/%d, %f%% loss: %0.2f'% (epoch+1, EPOCHS, i/len(train_iter)*100, loss.item()), end='')

Epoch: 10/10, 99.981259% loss: 1.35

In [0]:
def accuracy(y_actual, y_pred):
    return np.sum(y_actual == y_pred).astype('int') / y_actual.shape[0]

In [0]:
avg_acc = 0 
for i, batch in enumerate(test_iter):
        x, y = batch.Title, batch.Category
        loss, logit = model(input_ids=x.to(device), labels=y.to(device))
        _, y_pred_train = torch.max(logit, 1)
        acc = accuracy(y.cpu().numpy(), y_pred_train.cpu().detach().numpy().squeeze())
        avg_acc += acc
        print('\r%0.2f%% loss: %0.2f, accuracy %0.2f'% (i/len(valid_iter)*100, loss.item(), acc), end='')
print('\nAverage accuracy: ', avg_acc / len(test_iter))

99.85% loss: 1.54, accuracy 0.00
Average accuracy:  0.44302848575712145
